In [206]:
import numpy as np
import pandas as pd
import requests as re
import json
import os
import pickle
from functools import reduce
import random
from tqdm import tqdm

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_palette(sns.color_palette("Spectral"))
sns.set_style("darkgrid")

from matplotlib.ticker import StrMethodFormatter
# ax.xaxis.set_major_formatter(StrMethodFormatter("{x:.2f}"))

pd.options.display.float_format = '{:,}'.format

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
mpl.rcParams["axes.labelsize"] = 15
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 200)

In [2]:
"""
# CIK Lookup
# https://www.sec.gov/edgar/searchedgar/cik

# Google
cik = "0000320193"

headers = {'User-Agent': 'Exitwise@gmail.com', 'Accept-Encoding' : 'deflate'}

# Ebitda is non-GAAP
# First try to estimate ebitda from GAAP measures

# EBITDA = EBIT + DA

concepts = ["Revenues", "NetIncomeLoss", "DepreciationDepletionAndAmortization", "OperatingIncomeLoss"]
concept = concepts[-2]

# XBRL Get All Company Facts
company_facts = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"

# XBRL Get specific concept


# XBRL Frames (all filings for given concept) (Doesn't work for D&A)
frames = f"https://data.sec.gov/api/xbrl/frames/us-gaap/{concept}/USD/CY2019Q1I.json"


"""



In [2]:
file_path = 'ciks.pkl'

# Read ciks from pickle file
with open(file_path, 'rb') as file:
    ciks = pickle.load(file)

In [86]:
def x():
    return 1, 2

x()[0]

1

In [160]:
# Return tuple of dataframe of (date, concept_value) for a given cik and concept, company name
# Return float for dataframe only if invalid
def get_company_concept(cik, concept, debug = False, local = True):
	try:
		if local:
			with open(f'companyfacts/CIK{cik}.json', 'r') as json_file:
				data = json.load(json_file)
			if concept == "EntityPublicFloat":
				json_data = data['facts']['dei'][concept]['units']['USD']
			else:
				json_data = data['facts']['us-gaap'][concept]['units']['USD']

			# strip instantaneous tag from frame value	
			return pd.DataFrame([(i['frame'] if i['frame'][-1] != 'I' else i['frame'][:-1], i['val']) for i in json_data if 'frame' in i.keys()], columns=['Date', concept]), data['entityName']
		
		# API is slow, use local data
		else:
			api_url = f"https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/{concept}.json"
			headers = {'User-Agent': 'Exitwise@gmail.com', 'Accept-Encoding' : 'deflate'}
			response = re.get(api_url, headers = headers)
			# Check if the request was successful (status code 200)
			if response.status_code == 200:
				# Parse the JSON data
				json_data = response.json()

				if debug:
					print("JSON Data:")
					print(json.dumps(json_data, indent=2))  # Pretty print the JSON data
				
				return json_data
			else:
				print(f"Error: {response.status_code}")
	except KeyError:
		if debug:
			print(data['entityName'], cik)
			print(f"Error: {concept} not found\n")

		# only return float if can't find operatingincomeloss
		if concept == "OperatingIncomeLoss":
			return 0.0, data['entityName']
		else:
			return pd.DataFrame(columns=['Date', concept]), data['entityName']

cik = "0001504389"

# use entity public float to estimate market cap
# "The aggregate market value of the voting and non-voting common equity held by non-affiliates 
# computed by reference to the price at which the common equity was last sold, or the average bid 
# and asked price of such common equity, as of the last business day of the registrant's most recently completed second fiscal quarter.""

# EV = EntityPublicFloat - CashAndCashEquivalentsAtCarryingValue + (LongTermDebt + LiabilitiesCurrent)
# EBITDA = OperatingIncomeLoss + DepreciationDepletionAndAmortization
concepts = ["DepreciationDepletionAndAmortization", "OperatingIncomeLoss", "CashAndCashEquivalentsAtCarryingValue", "EntityPublicFloat", "LongTermDebt", "LiabilitiesCurrent", "CommonStockValue"]


data = get_company_concept(cik, concepts[3], local = True)[0]

data
    

,Date,EntityPublicFloat
0,CY2012Q2,0
1,CY2013Q2,0
2,CY2014Q2,0
3,CY2014Q3,12000000
4,CY2015Q3,64900
5,CY2016Q3,527878
6,CY2017Q3,1689940


In [207]:
batch = ciks

output = []
bad_ciks = []

for cik in tqdm(batch):
    dfs = []
    bad_cik = False
    for concept in concepts:
        try:
            data, name = get_company_concept(cik, concept, local = True)
            if type(data) == float:
                bad_cik = True
                break
            dfs.append(data)
        except:
            bad_ciks.append(cik)
            bad_cik = True
            break
    
    if bad_cik:
        continue

    df_merged = reduce(lambda left,right: pd.merge(left,right,on="Date", how='outer'), dfs)
    df_merged['cik'] = cik
    df_merged['name'] = name
    output.append(df_merged.sort_values(by='Date').reset_index(drop=True))

with open('bad_ciks.txt', 'w') as file:
    file.write('\n'.join(bad_ciks))


100%|██████████| 17687/17687 [20:27<00:00, 14.40it/s] 


In [208]:
df = pd.concat(output)

df = df.groupby(['cik', 'name', 'Date']).mean()

# drop rows where nan or zero for entity public float (public market cap) or Operating Income Loss (EBIT)
# Most companies only seem to report this value once per year, making quarterly data useless without questionable interpolation methods
df = df.loc[(df['EntityPublicFloat'] != 0) & (df['EntityPublicFloat'].notna())]
df = df.loc[(df['OperatingIncomeLoss'] != 0) & (df['OperatingIncomeLoss'].notna())]
df.fillna(0, inplace=True)

df["EBITDA"] = df["OperatingIncomeLoss"] + df["DepreciationDepletionAndAmortization"]
df["EV"] = df["EntityPublicFloat"] - df["CashAndCashEquivalentsAtCarryingValue"] + (df["LongTermDebt"] + df["LiabilitiesCurrent"])

def ev_ebitda(row):
    if row['EBITDA'] == 0:
        return 0
    else:
        return row["EV"] / row["EBITDA"]

df["EV/EBITDA"] = df.apply(ev_ebitda, axis = 1)

df.to_csv('cik_data.csv')

In [209]:
df

DepreciationDepletionAndAmortization  \
cik        name                        Date                                             
0000001800 ABBOTT LABORATORIES         CY2008Q2                                   0.0   
                                       CY2009Q2                                   0.0   
                                       CY2010Q2                                   0.0   
                                       CY2011Q2                                   0.0   
                                       CY2012Q2                                   0.0   
...                                                                               ...   
0001931055 Medinotec, Inc.             CY2022Q3                              19,809.0   
0001932244 LEVER GLOBAL CORPORATION    CY2022Q2                                   0.0   
0001937653 ZYMEWORKS INC.              CY2022Q2                                   0.0   
0001937993 CADRENAL THERAPEUTICS, INC. CY2022Q2                                   0.0   
0001949543 Sitio Royalties Corp.       CY2022Q2                                   0.0   

                                                 OperatingIncomeLoss  \
cik        name                        Date                            
0000001800 ABBOTT LABORATORIES         CY2008Q2      1,406,585,000.0   
                                       CY2009Q2      1,671,420,000.0   
                                       CY2010Q2      1,605,966,000.0   
                                       CY2011Q2      1,773,453,000.0   
                                       CY2012Q2        564,190,000.0   
...                                                              ...   
0001931055 Medinotec, Inc.             CY2022Q3             25,499.0   
0001932244 LEVER GLOBAL CORPORATION    CY2022Q2           -481,319.0   
0001937653 ZYMEWORKS INC.              CY2022Q2        -65,823,000.0   
0001937993 CADRENAL THERAPEUTICS, INC. CY2022Q2           -482,347.0   
0001949543 Sitio Royalties Corp.       CY2022Q2         52,896,000.0   

                                                 CashAndCashEquivalentsAtCarryingValue  \
cik        name                        Date                                              
0000001800 ABBOTT LABORATORIES         CY2008Q2                        3,796,586,000.0   
                                       CY2009Q2                        4,204,304,000.0   
                                       CY2010Q2                        4,321,702,000.0   
                                       CY2011Q2                        4,077,018,000.0   
                                       CY2012Q2                        7,052,776,000.0   
...                                                                                ...   
0001931055 Medinotec, Inc.             CY2022Q3                                    0.0   
0001932244 LEVER GLOBAL CORPORATION    CY2022Q2                            1,311,733.0   
0001937653 ZYMEWORKS INC.              CY2022Q2                                    0.0   
0001937993 CADRENAL THERAPEUTICS, INC. CY2022Q2                                    0.0   
0001949543 Sitio Royalties Corp.       CY2022Q2                                    0.0   

                                                EntityPublicFloat  \
cik        name                        Date                         
0000001800 ABBOTT LABORATORIES         CY2008Q2  78,771,016,188.0   
                                       CY2009Q2  70,195,399,310.0   
                                       CY2010Q2  69,683,328,714.0   
                                       CY2011Q2  79,091,612,753.0   
                                       CY2012Q2  98,004,683,803.0   
...                                                           ...   
0001931055 Medinotec, Inc.             CY2022Q3       3,662,854.0   
0001932244 LEVER GLOBAL CORPORATION    CY2022Q2      47,745,695.0   
0001937653 ZYMEWORKS INC.              CY2022Q2     306,000,000.0   
0001937993 CADRENAL THER

In [66]:
with open(f'companyfacts/CIK{cik}.json', 'r') as json_file:
			data = json.load(json_file)

In [64]:
concepts = ["DepreciationDepletionAndAmortization", "OperatingIncomeLoss", "CashAndCashEquivalentsAtCarryingValue", "EntityPublicFloat", "LongTermDebt", "LiabilitiesCurrent"]

In [51]:
from datetime import datetime

end_date_s = '2024-1-1'
start_date_s = '2015-1-1'

# date ranges
end_date = datetime.strptime(end_date_s, "%Y-%m-%d")
start_date = datetime.strptime(start_date_s, '%Y-%m-%d')


In [3]:
query_string

'https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json'